#### Execute this cell before running any code in this file...

In [122]:
# Import Javascript Libraries: Execute this cell...
from IPython.display import Javascript

display(Javascript("jupyter_lib.js"))

<IPython.core.display.Javascript object>

# Section 7: Beginnings of a Game

Goals:
 - Implement code that draws landscape.
 
We'll begin by attempting to draw the landscape. To begin, were going to create a game loop. A game loop is key to a game running, as otherwise your game is just a still frame. A game loop can be split into two parts:
 - 'update': Update the locations of all loaded objects in the game.
 - 'draw': Draw the current state of the game to the screen.
 
To begin, we will write some empty methods for doing this below.

In [62]:
%%javascript

function update(timeStep, gameState) {
    gameState.timeStep = timeStep;
}

function draw(canvas, painter, gameState) {
    // Clear the canvas...
    painter.fillStyle = "white"
    painter.fillRect(0, 0, canvas.width, canvas.height);
    
    // Example draw...
    painter.fillStyle = "black";
    painter.font = "30px Ariel";
    painter.fillText("Hello World!, Time Step: " + gameState.timeStep, 0, 30);
    painter.fillText("Mouse Pressed: " + gameState.mousePressed, 0, 60);
    painter.fillText("Mouse Location: " + gameState.mouseLocation, 0, 90);
}

// This function will get called over and over...
function gameLoop(timeStep, gameState) {
    update(timeStep, gameState);
    draw(gameState.canvas, gameState.painter, gameState);
}

element.makeBaseGame(gameLoop);

<IPython.core.display.Javascript object>

Now that we have a basic game loop setup, we need to write some code that represents game 'objects'.

We'll split them into two categories.
 - Blocks: Static, non-moving blocks that make up the platforms. These can't move, but are much faster to draw and update.
 - Entities: These can move, but are expensive.
 
Both these `GameObjects` provide these methods:
 - `update`: Update the object.
 - `draw`: Draw the object.
 
Since we are focused on drawing the map, we are only going to focus on implementing Blocks for now.

In [123]:
%%javascript

class GameBlock extends GameCollisionObject {
    update(timeStep, gameState) {
        // Do we do anything in update?
        // No, because blocks don't move.
    }
    
    draw(canvas, painter, camera) {
        // Implement this:
        let [cx, cy, cw, ch] = camera.transformBox([this.x * this._blockSize , this.y * this._blockSize, this._blockSize, this._blockSize]);
                
        painter.fillStyle = "rgba(255, 0, 0, 0.5)";
        painter.fillRect(cx, cy, cw, ch);
    }
    
    drawPreview(canvas, painter, box) {
        let [cx, cy, cw, ch] = box;
        
        painter.fillStyle = "red";
        painter.fillRect(cx, cy, cw, ch);
    }
}

class Player extends GameCollisionObject {
    constructor(x, y, blockSize, sprites) {
        super(x, y, blockSize, sprites);
        this._sprite = sprites.playerSprite.buildSprite();
        this._sprite.setAnimation("stand");
        this._inset = [8 / 32, 2 / 32, 16 / 32, 27 / 32];
        this._vx = 0; 
        this._vy = 0;
        this._ax = 0;
        this._ay = 0.09 / 1000;
        this._priorUp = false;
        this._numJumps = Infinity;
        this._fric_fac = 0.95;
        
        this._movable = true;
    }
    
    update(timeStep, gameState) {
        let keys = gameState.keysPressed;
        
        let anim = "stand"
        
        this._ax = 0;
        if("ArrowLeft" in keys) {
            this._ax -= 0.05 / 1000;
            anim = "run";
        }
        if("ArrowRight" in keys) {
            this._ax += 0.05 / 1000;
            anim = "run";
        }
        if(!this._priorUp && ("ArrowUp" in keys) && (this._numJumps > 0)) {
            this._vy = -5 / 1000 * timeStep;
            this._numJumps--;
            anim = "jump";
        }
        this._priorUp = "ArrowUp" in keys;
        
        if(this._sprite.getAnimation() != "jump") this._sprite.setAnimation(anim);
                
        this._vx += this._ax * timeStep;
        this._vx *= this._fric_fac;
        this._vy += this._ay * timeStep;
        this._vy *= this._fric_fac;
        
        this.x += this._vx * timeStep;
        this.y += this._vy * timeStep;
        
        if((this.y + this._inset[3]) >= (gameState.level.numChunks[1] * gameState.level.chunkSize)) return true;
                
        this._sprite.update(timeStep);
    }
    
    draw(canvas, painter, camera) {
        // Implement this:
        let [xi, yi, wi, hi] = this._inset;
        let [inx, iny] = [
            (this.x * this._blockSize) - (xi * this._blockSize), 
            (this.y * this._blockSize) - (yi * this._blockSize)
        ];
        let [cx, cy, cw, ch] = camera.transformBox([inx, iny, this._blockSize, this._blockSize]);
        this._sprite.draw(painter, cx, cy, cw, ch);
    }
    
    drawPreview(canvas, painter, box) {
        let [cx, cy, cw, ch] = box;        
        this._sprite.draw(painter, cx, cy, cw, ch);
    }
    
    handleCollisions(objList) {        
        for(let [obj, side] of objList) {
            if(side == "bottom") {
                this._numJumps = 2;
                if(this._sprite.getAnimation() == "jump") this._sprite.setAnimation("stand");
            }
        }
    }
    
    getHitBox() {
        let [xi, yi, wi, hi] = this._inset;
        return [this.x, this.y, wi, hi];
    }
}


class Enemy extends GameCollisionObject {
    constructor(x, y, blockSize, sprites) {
        super(x, y, blockSize, sprites);
        this._sprite = sprites.enemySprite.buildSprite();
        this._sprite.setAnimation("run");
        this._vx = 0;
        this._vy = 0;
        this._ax = 0;
        this._ay = 0.09 / 1000;
        this._fric_fac = 0.95;
        this._inset = [8 / 32, 2 / 32, 16 / 32, 27 / 32];
        
        this._movable = true;
    }
    
    update(timeStep, gameState) {
        this._sprite.update(timeStep);
        let [px, py] = gameState.__player.getLocation();
        
        this._ax = 0.05 / 1000 * Math.sign(px - this.x);
        
        this._vx += this._ax * timeStep;
        this._vx *= this._fric_fac;
        this._vy += this._ay * timeStep;
        this._vy *= this._fric_fac;
        
        this.x += this._vx * timeStep;
        this.y += this._vy * timeStep;
    }
    
    draw(canvas, painter, camera) {
        // Implement this:
        let [xi, yi, wi, hi] = this._inset;
        let [inx, iny] = [
            (this.x * this._blockSize) - (xi * this._blockSize), 
            (this.y * this._blockSize) - (yi * this._blockSize)
        ];
        let [cx, cy, cw, ch] = camera.transformBox([inx, iny, this._blockSize, this._blockSize]);
        this._sprite.draw(painter, cx, cy, cw, ch);
    }
    
    handleCollisions(objList) {
        for(let [obj, side] of objList) {
        }
    }
    
    drawPreview(canvas, painter, box) {
        let [cx, cy, cw, ch] = box;        
        this._sprite.draw(painter, cx, cy, cw, ch);
    }
    
    getHitBox() {
        let [xi, yi, wi, hi] = this._inset;
        return [this.x, this.y, wi, hi];
    }
}

function draw(canvas, painter, gameState) {
    // Clear the canvas...
    painter.fillStyle = "white"
    painter.fillRect(0, 0, canvas.width, canvas.height);
    
    // Draw our single block...
    gameState.block.draw(gameState.canvas, gameState.painter, gameState.camera);
}

// This function will get called over and over...
function gameLoop(timeStep, gameState) {
    update(timeStep, gameState);
    draw(gameState.canvas, gameState.painter, gameState);
}

let gameState = {
    "block": new GameBlock(0, 0, 32),
    "cameraMoveSpeed": 5
}

let levelData = {
    sprites: {
        playerSprite: {
            image: "images/player.png",
            animations: {
                "stand": {
                    "frames": [0],
                },
                "run": {
                    "frames": [0, 1, 2, 3],
                    "speed": 100
                },
                "jump": {
                    "frames": [0, 4, 5],
                    "speed": 150,
                    "cycles": 1
                }
            }
        },
        enemySprite: {
            image: "images/enemy.png",
            animations: {
                "stand": {
                    "frames": [0],
                },
                "run": {
                    "frames": [0, 1, 2, 3],
                    "speed": 100
                },
                "jump": {
                    "frames": [0, 4, 5, 5, 4],
                    "speed": 150,
                    "cycles": 1
                }
            }
        }
    }
}

element.makeGame("testdata/test_level.json", [GameBlock], [Enemy], levelData, Player);

<IPython.core.display.Javascript object>

Additional Steps:
 - Great, you got it working! But, eventually we will want to be able to move around the screen. To do this, you should transform the coordinates of the box using the camera before plotting it to the screen (hint: `transformBox` method of camera might help here) The camera converts points in the virtual game space into the physical canvas space that we see.
 
 - Replace your update function with the code below. What is it doing? Is it moving the block, or our perspective of it?
```javascript
function update(timeStep, gameState) {
    let keys = gameState.keysPressed;
    let c = gameState.camera;
    let [cx, cy] = c.getCenterPoint();
    let vel = 0.1 * timeStep;
        
    if("ArrowUp" in keys || "KeyW" in keys) cy += vel;
    if("ArrowDown" in keys || "KeyS" in keys) cy -= vel;
    if("ArrowLeft" in keys || "KeyA" in keys) cx += vel;
    if("ArrowRight" in keys || "KeyD" in keys) cx -= vel;
    
    c.setCenterPoint([cx, cy]);
    
    c.update();
}
```
   instead of moving the camera like this manually, we will eventually set it up to track the player...
   
As you may have noticed already, the makeGame method adds some properties to the gameState object for general use. These include:
 - `lastTimeStamp`: Last time game was updated before this update.
 - `keepRunning`: Boolean flag to indicate if game should keep looping. Should be set to false to stop the game.
 - `canvas`: The canvas.
 - `painter`: The canvas painter or 2D rendering context.
 - `keysPressed`: A object which stores the key codes of all keys currently pressed down. You will use this for handling input.
 - `camera`: The camera created for the game. Converts game coordinates into screen coordinates, and can track an object.
 - `mousePressed`: If the mouse button is pressed down.
 - `mouseLocation`: The location of the mouse.
 
You will several of these quite often in your code.